Transfer Learning Xception

#  Sexual Harassment Detection
Sexual Harassment Detection Using a Transfer Learning method Xception

## Importing necessary modules
### Data Processing and Manipulation
cv2 : Used for image processing tasks<br>
glob : Used to find file paths matching a pattern<br>
os : Used for interacting with the operating system<br>
numpy : Used for numerical computations and array handling<br>
pandas : Used for data manipulation and analysis<br>
### For Visualization
matplotlib.pyplot : Used for plotting graphs and images<br>
tqdm : Used to display progress bars during iterations<br>
### For Building ML model
keras : High-level neural networks API for building and training models<br>
tensorflow : Deep Learning framework used as backend for Keras<br>
sklearn : Library for machine learning tasks, such as model evaluation


In [1]:
import tensorflow as tf
import os
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

if not os.path.isdir('models'):
    os.mkdir('models')
    
print('TensorFlow version:', tf.__version__)
print('Is using GPU?', tf.test.is_gpu_available())

TensorFlow version: 2.18.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is using GPU? False


In [2]:
import cv2
from glob import glob
from scipy import stats as s
import keras
from keras.models import Sequential
import tensorflow.keras.applications.xception
from keras.applications.xception import Xception
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.layers import Flatten

### Reading a CSV file
This csv file contains two columns which has the location of the images in our dataset and the class label (0-Healthy Environment / 1-Harassment Detected) associated with it.

The csv file - 'harassment_as_csv.csv' is generated as an output of 'preprocessing.ipynb'
#### Loading 
Reads the csv file using pandas and stores it in the 'train' DataFrame.

In [3]:
train = pd.read_csv(r"frames_info.csv")
train.head()

,video_file,frame_filename,label
0,vid11.mp4,preprocessed_imgs\vid11_frame_0000.jpg,1
1,vid11.mp4,preprocessed_imgs\vid11_frame_0001.jpg,1
2,vid11.mp4,preprocessed_imgs\vid11_frame_0002.jpg,1
3,vid11.mp4,preprocessed_imgs\vid11_frame_0003.jpg,1
4,vid11.mp4,preprocessed_imgs\vid11_frame_0004.jpg,1


## Preprocessing Images 
This iterates through each row of the 'train' DataFrame, loads the image specified by the 'image' column.
#### Preprocessing Steps:
The image is resized to (224,224,3)

Converts it to a NumPy array.

Normalizes the pixel values to the range [0,1]

These processed images are then stored in the 'train_image' list.
This list is then converted to a NumPy array 'X'

In [4]:
# Import the load_img function from tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [11]:

# Creating empty lists
train_image = []
train_labels = []

# For loop to read and store frames
for i in tqdm(range(train.shape[0])):
    file_path = train['frame_filename'][i]
    label = train['label'][i]
    
    # Check if the file exists
    if os.path.exists(file_path):
        try:
            # Loading the image and resizing to (224, 224)
            img = load_img(file_path, target_size=(224, 224))
            # Converting to array
            img = img_to_array(img)
            # Normalizing the pixel value
            img = img / 255.0
            # Appending the image and label
            train_image.append(img)
            train_labels.append(label)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

# Converting the lists to numpy arrays
X = np.array(train_image)
y = np.array(train_labels)

# Shape of the array
print("Shape of image array:", X.shape)
print("Shape of labels array:", y.shape)


100%|██████████| 3523/3523 [00:05<00:00, 665.48it/s] 


Shape of image array: (2970, 224, 224, 3)
Shape of labels array: (2970,)


#### Separating Target
Extracts the 'class' column from the 'train' DataFrame, which contains the class labels of the images.

#### Dataset Segregation
The train_test_split function splits the 'x' and 'y' into training and testing parts.
The split is done with test size of 20% .

The 'stratify' parameter ensures that the class distribution is preserved in both the sets.

In [12]:

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)

The function 'get_dummies' converts the categorical class labels into one-hot enocoded format.

In [13]:
# creating dummies of target variable for train and validation set
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

## Loading our Pre-trained Model
A pre-trained VGG16 model is loaded from keras with pre-trained weights on the ImageNet dataset.
The fully connected layers at the top of the network is excluded by setting the parameter 'include_top' to False

In [14]:
# creating the base model of pre-trained Xception model
base_model = Xception(weights='imagenet', include_top=False, input_shape=(224,224,3))

### Preprocessing
preprocessing the training and validation images

In [15]:
# Preprocess the training and validation images
X_train_preprocessed = keras.applications.xception.preprocess_input(X_train)
X_test_preprocessed = keras.applications.xception.preprocess_input(X_test)

### Extracting Features
Here we extract features from the images in the training and testing sets using our base_model - Xception.
The 'predict' method is applied to both sets to obtain feature vectors for each image.

In [16]:
# extracting features for training frames
X_train_features = base_model.predict(X_train_preprocessed)

# extracting features for validation frames
X_test_features = base_model.predict(X_test_preprocessed)

# flatten the feature arrays
X_train_flatten = X_train_features.reshape(X_train_features.shape[0], -1)
X_test_flatten = X_test_features.reshape(X_test_features.shape[0], -1)

print("Shape of X_train_flatten:", X_train_flatten.shape)
print("Shape of X_test_flatten:", X_test_flatten.shape)

75/75 ━━━━━━━━━━━━━━━━━━━━ 133s 2s/step
19/19 ━━━━━━━━━━━━━━━━━━━━ 22s 1s/step
Shape of X_train_flatten: (2376, 100352)
Shape of X_test_flatten: (594, 100352)


In [17]:
# extracting features for training frames
X_train = base_model.predict(X_train)
X_train.shape

75/75 ━━━━━━━━━━━━━━━━━━━━ 166s 2s/step


(2376, 7, 7, 2048)

In [18]:
# extracting features for validation frames
X_test = base_model.predict(X_test)
X_test.shape

19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 1s/step


(594, 7, 7, 2048)

The extracted features are reshaped into a format suitable for feeding into the subsequent fully connected layers. This depends on the architecture of the VGG16 model.

In [24]:
# reshaping the training as well as validation frames in single dimension
X_train = X_train.reshape(2376,7*7*2048)
X_test = X_test.reshape(594,7*7*2048)

The feature vectors are normalized by dividing them by the maximum value of the training set.

In [25]:
# normalizing the pixel values
max = X_train.max()
X_train = X_train/max
X_test = X_test/max
X_train.shape

(2376, 100352)

## Model Architecture
Fully Connected(Dense) layers with ReLU activation function.

Dropout Layers for regularization.

Dense Layer with softmax activation for classification.

In [34]:
#defining the model architecture
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(7*7*2048,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

c:\Users\nikhi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


#### Callback Definition 
A callback to save the model weights during training. It is saved in the 'weight.hdf5' file whenever the validation loss improves.

In [30]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.keras', save_best_only=True, monitor='val_loss', mode='min')

#### Model compilation
The model is compiled with a categorical cross-entropy loss function, Adam optimizer, and accuracy metric. 

In [31]:
# compiling the model
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │   102,761,472 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 2)              │           258 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 103,450,754 (394.63 MB)

 Trainable params: 103,450,754 (394.63 MB)

 Non-trainable params: 0 (0.00 B)

# Graphs,Plots and Visualization

## Sample Images
Plot of few sample images from the dataset along with their class labels

In [32]:
# training the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=256)

Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 21s 2s/step - accuracy: 0.5307 - loss: 0.7119 - val_accuracy: 0.5758 - val_loss: 0.6701
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 31s 3s/step - accuracy: 0.5573 - loss: 0.7095 - val_accuracy: 0.5758 - val_loss: 0.6576
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - accuracy: 0.5957 - loss: 0.6662 - val_accuracy: 0.6936 - val_loss: 0.6146
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.6533 - loss: 0.6121 - val_accuracy: 0.9226 - val_loss: 0.4221
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.8369 - loss: 0.4453 - val_accuracy: 0.9529 - val_loss: 0.1695
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - accuracy: 0.9120 - loss: 0.2489 - val_accuracy: 0.9781 - val_loss: 0.0714
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9591 - loss: 0.1156 - val_accuracy: 0.9865 - val_loss: 0.0577
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.9619 - loss: 0.0972 - val_accuracy: 1.0000 - val_loss:

In [33]:
scores = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {scores[1]*100} ")

19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 1.0000 - loss: 0.0037
Test Accuracy: 100.0 
